In [ ]:
import tensorflow as tf
import numpy as np

#model parameters
convWidth = 5
conv1feat = 64
conv2feat = 64
conv3feat = 64
linear1feat = 32



def weight_variable(shape, sd=0.0001):
    initial = tf.truncated_normal(shape, sd)
    return tf.Variable(initial)


def bias_variable(shape, value=0.0001):
    initial = tf.constant(value, shape=shape, dtype=tf.float32)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')


def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')


class model:
    def __init__(self, imagesIn, labelsTru):
        
        # get dimensions
        dimIn = int(images.get_shape()[1])
        classes = int(labelsTru.get_shape()[1])
        
        self.images = imagesIn
        
        # define graph variables
        Wconv1 = weight_variable([convWidth, convWidth, 3, conv1feat], 0.0001) 
        bConv1 = bias_variable([conv1feat])
        Wconv2 = weight_variable([convWidth, convWidth, conv1feat, conv2feat], 0.01) 
        bConv2 = bias_variable([conv2feat])
        Wconv3 = weight_variable([convWidth, convWidth, conv2feat, conv3feat], 0.01) 
        bConv3 = bias_variable([conv3feat])
        dim3 = dimIn /2 /2 
        Wfc1 = weight_variable([dim3 * dim3 * conv3feat, linear1feat], 0.1) 
        bFC1 = bias_variable([linear1feat])
        Wfc2 = weight_variable([linear1feat, classes], 0.1)
        bFC2 = bias_variable([classes])
        
        # convolution 1, with batch-norm and pooling
        hConv1pre = conv2d(imagesIn, Wconv1) + bConv1
        hConv1n = tf.contrib.layers.batch_norm(hConv1pre, center=True, scale=True)
        hConv1n = tf.nn.relu(hConv1n)
        # hConv1n = tf.nn.dropout(hConv1n,1-hiddenDropRate)
        hPool1 = max_pool(hConv1n)
        # hPool1 = hConv1

        # convolution 2, with batch-norm and no pooling
        hConv2pre = conv2d(hPool1, Wconv2) + bConv2
        hConv2n = tf.contrib.layers.batch_norm(hConv2pre, center=True, scale=True)
        hConv2n = tf.nn.relu(hConv2n)
        # hConv2n = tf.nn.dropout(hConv2n, 1-hiddenDropRate)
        # hPool2 = max_pool(hConv2n)
        hPool2 = hConv2n

        # convolution3 with skip connection to output from conv1, followed by pooling
        hConv3pre = conv2d(hPool2, Wconv3) + bConv3 + hPool1
        hConv3n = tf.contrib.layers.batch_norm(hConv3pre, center=True, scale=True)
        hConv3n = tf.nn.relu(hConv3n)
        hPool3 = max_pool(hConv3n)

        # non-linear layer 1, flatten processed image into vector; transform and apply batch norm and relu
        hPool2flat = tf.reshape(hPool3, [-1,dim*dim*conv2feat])
        hFC1pre = tf.matmul(hPool2flat, Wfc1) + bFC1
        hFC1n = tf.contrib.layers.batch_norm(hFC1pre, center=True, scale=True)
        hFC1n = tf.nn.relu(hFC1n)

        # readout layer, softmax means elements of y form probability distribution (sum to one)
        self.labels = tf.nn.softmax(tf.matmul(hFC1n, Wfc2) + bFC2)
        
        # calculate cross entropy between predictions and ground truth
        self.crossEntropy = tf.reduce_mean(-tf.reduce_sum(labelsTru * tf.log(self.y), reduction_indices=[1]))
        
        correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(labelsTru, 1))
        
        # correct_prediction gives list of booleans, take mean to measure % accuracy
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def use_nesterov(self, eta=0.1, gamma=0.9):
        # set training method
        self.trainStep = tf.train.MomentumOptimizer(eta, gamma, use_nesterov=True).minimize(self.crossEntropy)


In [1]:
!wget https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADSNUu0bseoCKuxuI5ZeTl1a/Img?dl=0&preview=img_align_celeba.zip
    